By: 10117094	Muhammad Walidin
    10116476	Idham Rahadian
    10116490	Muhammad Fajar Maulana

In [1]:
import pandas as pd
import numpy as np
import re
import itertools
import collections
import string
from collections import OrderedDict

In [2]:
raw = pd.read_csv('data/data_raw_clean.csv')

In [3]:
stop_words = np.array(pd.read_csv("data/external/stopwords_ID.txt",
                        sep="\n", header=None).values)
neg_words = np.array(pd.read_csv("data/external/negative_keyword_ID.txt",
                        sep="\n", header=None).values)
pos_words = np.array(pd.read_csv("data/external/positive_keyword_ID.txt",
                        sep="\n", header=None).values)
slang_words = pd.read_csv("data/external/kbba_ID.txt",
                        sep="\t", header=None)
root_words = np.array(pd.read_csv("data/external/rootword_ID.txt",
                        sep="\n", header=None).values)
slang = pd.read_csv("data/external/slangword_ID.txt",
                        sep=":", header=None)
emoticon = pd.read_csv("data/external/emoticon.txt",
                        sep="\t", header=None)
booster_words = np.array(pd.read_csv("data/external/boosterword_ID.txt",
                        sep="\n", header=None).values)
baku_words = pd.read_csv("data/external/katabaku_ID.txt",
                        sep="|", header=None)
baku_words.columns = [1,0]

In [4]:
slang_words = pd.concat([slang_words, slang, baku_words])
sentiment_words = np.concatenate((pos_words, neg_words, booster_words))
slang_words.drop_duplicates(inplace=True)
emoticon.drop_duplicates(inplace=True)
emoticon = dict(zip(emoticon[0], emoticon[1]))
slang_words = dict(zip(slang_words[0],slang_words[1]))
neg_words = np.unique(neg_words)
pos_words = np.unique(pos_words)
stop_words = np.unique(stop_words)
stop_words = [word for word in stop_words if word not in sentiment_words]

In [5]:
print("------Dictionary Info---------")
print("Slang words = "  + str(len(slang_words)) + " entries")
print("Emoticon = "  + str(len(emoticon)) + " entries" )
print("Root words = "  + str(len(root_words)) + " entries")
print("Sentiment words = "  + str(len(sentiment_words)) + " entries")

------Dictionary Info---------
Slang words = 2398 entries
Emoticon = 110 entries
Root words = 27979 entries
Sentiment words = 8449 entries


In [6]:
print(raw.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   rating  1682 non-null   int64 
 1   review  1682 non-null   object
dtypes: int64(1), object(1)
memory usage: 26.4+ KB
None


# Hapus data duplikasi

In [7]:
raw = raw.dropna(subset=['review'],how='all')
raw = raw.drop_duplicates(subset=['review'])

In [8]:
print(raw.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 807 entries, 0 to 1681
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   rating  807 non-null    int64 
 1   review  807 non-null    object
dtypes: int64(1), object(1)
memory usage: 18.9+ KB
None


# Siapkan helper untuk cleaning data

In [9]:
satuan = ['', 'satu', 'dua', 'tiga', 'empat', 'lima', 'enam', 'tujuh',
          'delapan', 'sembilan', 'sepuluh', 'sebelas']
    
def terbilang_(n):
    if n >= 0 and n <= 11:
        hasil = [satuan[n]]
    elif n >= 12 and n <= 19:
        hasil = terbilang_(n % 10) + ['belas']
    elif n >= 20 and n <= 99:
        hasil = terbilang_(n // 10) + ['puluh'] + terbilang_(n % 10)
    elif n >= 100 and n <= 199:
        hasil = ['seratus'] + terbilang_(n - 100)
    elif n >= 200 and n <= 999:
        hasil = terbilang_(n // 100) + ['ratus'] + terbilang_(n % 100)
    elif n >= 1000 and n <= 1999:
        hasil = ['seribu'] + terbilang_(n - 1000)
    elif n >= 2000 and n <= 999999:
        hasil = terbilang_(n // 1000) + ['ribu'] + terbilang_(n % 1000)
    elif n >= 1000000 and n <= 999999999:
        hasil = terbilang_(n // 1000000) + ['juta'] + terbilang_(n % 1000000)
    else:
        hasil = terbilang_(n // 1000000000) + ['milyar'] + terbilang_(n % 100000000)
    return hasil

def terbilang(n):
    if n == 0:
        return 'nol'
    t = terbilang_(n)
    while '' in t:
        t.remove('')
    return ' '.join(t)

def translate_angka(review):
    number = re.findall("[0-9]+",review)
    for digit in number:
        review = re.sub(digit,terbilang(int(digit))+" ",review)
    return review

In [10]:
def translate_repeating_words(review):
    repeating_words = re.findall(r'\w*(?:2|")',review)
    for word in repeating_words:
        cleaned = word[:-1]
        review = re.sub(word,cleaned + " " + cleaned, review)
    return review

In [11]:
def delete_suffix_nya(review):
    return re.sub("(?:nya|ny)[$|\s]"," ",review)

In [12]:
def translate_emoticon(t):
    for w, v in emoticon.items():
        pattern = re.compile(re.escape(w))
        match = re.search(pattern,t)
        if match:
            t = re.sub(pattern,v,t)
    return t

In [13]:
def translate_non_alpha_num(t):
    non_alpha_num = {
        '%' : 'persen',
        '$' : 'dolar',
        '@' : 'di',
        '&' : 'dan',
        '/' : 'atau',
        '+' : 'plus'
    }
    for w, v in non_alpha_num.items():
        pattern = re.compile(re.escape(w))
        match = re.search(pattern,t)
        if match:
            t = re.sub(pattern,v + " ",t)
    return t

In [14]:
def remove_non_alphanumeric(review):
    return re.sub("[^a-zA-Z\d]"," ", review)

In [15]:
def normalizing_words(review):
    return ''.join(''.join(s)[:1] for _, s in itertools.groupby(review))

# Cleansing data dari Helper

In [16]:
raw['review'] = raw['review'].apply(translate_repeating_words)
raw.head()

,rating,review
0,5,Tidak ada review
1,5,"Puas, kecil tapi kencang."
3,1,Barang yang datang bukan baracuda 3.5 7 700rpm...
4,5,"Thanks barang sudah sampai, sehari langsung sa..."
12,5,Barang bagus pengirman cepat. mantab deh


In [17]:
raw['review'] = raw['review'].apply(translate_emoticon)
raw.head()

,rating,review
0,5,Tidak ada review
1,5,"Puas, kecil tapi kencang."
3,1,Barang yang datang bukan baracuda 3.5 7 700rpm...
4,5,"Thanks barang sudah sampai, sehari langsung sa..."
12,5,Barang bagus pengirman cepat. mantab deh


In [18]:
raw['review'] = raw['review'].apply(lambda x: x.lower())
raw.head()

,rating,review
0,5,tidak ada review
1,5,"puas, kecil tapi kencang."
3,1,barang yang datang bukan baracuda 3.5 7 700rpm...
4,5,"thanks barang sudah sampai, sehari langsung sa..."
12,5,barang bagus pengirman cepat. mantab deh


In [19]:
raw['review'] = raw['review'].apply(translate_non_alpha_num)
raw.head()

,rating,review
0,5,tidak ada review
1,5,"puas, kecil tapi kencang."
3,1,barang yang datang bukan baracuda 3.5 7 700rpm...
4,5,"thanks barang sudah sampai, sehari langsung sa..."
12,5,barang bagus pengirman cepat. mantab deh


In [20]:
raw['review'] = raw['review'].apply(remove_non_alphanumeric)
raw.head()

,rating,review
0,5,tidak ada review
1,5,puas kecil tapi kencang
3,1,barang yang datang bukan baracuda 3 5 7 700rpm...
4,5,thanks barang sudah sampai sehari langsung sa...
12,5,barang bagus pengirman cepat mantab deh


In [21]:
raw['review'] = raw['review'].apply(delete_suffix_nya)
raw['review']

0                                        tidak ada review
1                               puas  kecil tapi kencang 
3       barang yang datang bukan baracuda 3 5 7 700rpm...
4       thanks barang sudah sampai  sehari langsung sa...
12               barang bagus pengirman cepat  mantab deh
                              ...                        
1675    barang kondisi ok  respon cepat dan  sangat baik 
1677    barang sampai tepat waktu sesuai target lazada...
1678    pelayanan sangat bagus  barang sesuai dengan y...
1679    barang sampai lebih cepat dan dalam kondisi sa...
1681                                bonus  bohong gak ada
Name: review, Length: 807, dtype: object

In [22]:
raw['review'] = raw['review'].apply(translate_angka)
raw.head()

,rating,review
0,5,tidak ada review
1,5,puas kecil tapi kencang
3,1,barang yang datang bukan baracuda tiga lima ...
4,5,thanks barang sudah sampai sehari langsung sa...
12,5,barang bagus pengirman cepat mantab deh


In [23]:
raw['review'] = raw['review'].apply(normalizing_words)
raw.head()

,rating,review
0,5,tidak ada review
1,5,puas kecil tapi kencang
3,1,barang yang datang bukan baracuda tiga lima tu...
4,5,thanks barang sudah sampai sehari langsung sam...
12,5,barang bagus pengirman cepat mantab deh


In [24]:
raw['review'] = raw['review'].apply(lambda x: x.split())
raw['review'] = raw['review'].apply(lambda x: np.array(x))
def mapping_slang_words(review):
    return [slang_words[word] if word in slang_words else word for word in review]
raw['review'] = raw['review'].apply(mapping_slang_words)
raw.head()

,rating,review
0,5,"[tidak, ada, ulasan]"
1,5,"[puas, kecil, tapi, kencang]"
3,1,"[barang, yang, datang, bukan, baracuda, tiga, ..."
4,5,"[terima kasih, barang, sudah, sampai, sehari, ..."
12,5,"[barang, bagus, pengirman, cepat, mantap, deh]"


In [28]:
raw = raw.dropna(subset=['review'],how='all')
raw = raw[raw['review'].map(len) > 0]
def avg_word(words):
  return (sum(len(word) for word in words)/len(words))

raw['avg_word'] = raw['review'].apply(lambda x: avg_word(x))
raw['word_count'] = raw['review'].map(len)

In [29]:
raw

,rating,review,avg_word,word_count
0,5,"[tidak, ada, ulasan]",4.666667,3
1,5,"[puas, kecil, tapi, kencang]",5.000000,4
3,1,"[barang, yang, datang, bukan, baracuda, tiga, ...",5.000000,13
4,5,"[terima kasih, barang, sudah, sampai, sehari, ...",6.176471,17
12,5,"[barang, bagus, pengirman, cepat, mantap, deh]",5.666667,6
...,...,...,...,...
1675,5,"[barang, kondisi, oke, respon, cepat, dan, san...",5.000000,8
1677,5,"[barang, sampai, tepat, waktu, sesuai, target,...",5.318182,22
1678,5,"[pelayanan, sangat, bagus, barang, sesuai, den...",5.700000,20
1679,5,"[barang, sampai, lebih, cepat, dan, dalam, kon...",4.888889,18


In [30]:
raw.to_csv("data/data/1.0_data_review_clean_without_stop_removal.csv", index=False)